In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse
os.environ["DATASET"] = "../datasets"

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
# logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [3]:
parser = argparse.ArgumentParser(description='Pretrain or fine-tune models for NLP tasks.')
parser.add_argument('--command', default="evaluate", help='Command: train/evaluate/train-and-evaluate')
parser.add_argument('--conf_files', nargs='+', help='Path(s) to the config file(s).')
parser.add_argument('--user_dir', help='Path to the user defined module for tasks (models, criteria), optimizers, and lr schedulers.')
parser.add_argument('--config_overrides', nargs='*', help='Override parameters on config with a json style string, e.g. {"<PARAM_NAME_1>": <PARAM_VALUE_1>, "<PARAM_GROUP_2>.<PARAM_SUBGROUP_2>.<PARAM_2>": <PARAM_VALUE_2>}. A key with "." updates the object in the corresponding nested dict. Remember to escape " in command line.')
parser.add_argument('--overrides', help='arguments that used to override the config file in cmdline', nargs=argparse.REMAINDER)

cmdline_args = parser.parse_args('')
# cmdline_args.conf_files = [os.path.join(home_dir, "configs/xdecoder/svlp_focalt_lang.yaml")]
cmdline_args.conf_files = [os.path.join(home_dir, "configs/hdecoder/vcoco.yaml")]
cmdline_args.overrides = ['WEIGHT', 'true', 'RESUME_FROM', '../checkpoints/xdecoder_focalt_best_openseg.pt'] 

opt = load_opt_from_config_files(cmdline_args.conf_files)
opt["base_path"] = "../"

keys = [cmdline_args.overrides[idx*2] for idx in range(len(cmdline_args.overrides)//2)]
vals = [cmdline_args.overrides[idx*2+1] for idx in range(len(cmdline_args.overrides)//2)]
print(keys, vals)
vals = [val.replace('false', '').replace('False','') if len(val.replace(' ', '')) == 5 else val for val in vals]
types = []
for key in keys:
    key = key.split('.')
    ele = opt.copy()
    while len(key) > 0:
        ele = ele[key.pop(0)]
    types.append(type(ele))

config_dict = {x:z(y) for x,y,z in zip(keys, vals, types)}

load_config_dict_to_opt(opt, config_dict)
for key, val in cmdline_args.__dict__.items():
    if val is not None:
        opt[key] = val
opt = init_distributed(opt)

['WEIGHT', 'RESUME_FROM'] ['true', '../checkpoints/xdecoder_focalt_best_openseg.pt']


In [4]:
pretrained_pth = os.path.join(opt['RESUME_FROM'])
print(pretrained_pth)

../checkpoints/xdecoder_focalt_best_openseg.pt


In [5]:
model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).eval().cuda()

*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear1.bias, Model Shape: torch.Size([2048])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear1.weight, Model Shape: torch.Size([2048, 512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear2.bias, Model Shape: torch.Size([512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.linear2.weight, Model Shape: torch.Size([512, 2048])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.in_proj_bias, Model Shape: torch.Size([1536])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.in_proj_weight, Model Shape: torch.Size([1536, 512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.out_proj.bias, Model Shape: torch.Size([512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.multihead_attn.out_proj.weight, Model Shape: torch.Size([512, 512])
*UNLOADED* hoid_head.hoi_decoder.hopd_decoder.layers.0.norm1.bias, Model Shape: torch.Size([512])
*UNLOAD

In [6]:
from datasets import build_evaluator, build_eval_dataloader
dataloaders = build_eval_dataloader(opt)
dataset_names = opt["DATASETS"]["TEST"]
opt['GRADIENT_ACCUMULATE_STEP'] = int(opt.get('GRADIENT_ACCUMULATE_STEP', 1))
opt['LR_SCHEDULER_PARAMS'] = opt.get('LR_SCHEDULER_PARAMS', {})

In [7]:
def get_dataloaders(
    dataset_label: str,
    is_evaluation: bool
):
    if is_evaluation:

        dataloaders = build_eval_dataloader(opt)
        dataloader = dataloaders[0]

        # temp solution for lr scheduler
        steps_total = len(dataloader)
        steps_acc = opt['GRADIENT_ACCUMULATE_STEP']
        steps_update = steps_total // steps_acc
        opt["LR_SCHEDULER_PARAMS"]["steps_update_per_epoch"] = steps_update
    return dataloader

In [8]:
from trainer import HDecoder_Trainer as Trainer
if cmdline_args.user_dir:
    absolute_user_dir = os.path.abspath(cmdline_args.user_dir)
    opt['base_path'] = absolute_user_dir
trainer = Trainer(opt)
trainer.eval()

----------------
MPI Adapter data
----------------
environment info: no MPI
init method url: tcp://127.0.0.1:36873
world size: 1
local size: 1
rank: 0
local rank: 0
master address: 127.0.0.1
master port: 36873
----------------


Evaluation start ...
[{'id': 0, 'file_name': '../datasets/v-coco/images/val2014/COCO_val2014_000000565248.jpg', 'hoi_annotation': [{'subject_id': 1, 'category_id': 2, 'object_id': 0}, {'subject_id': 2, 'category_id': 2, 'object_id': 3}, {'subject_id': 1, 'category_id': 3, 'object_id': 0}, {'subject_id': 2, 'category_id': 3, 'object_id': 3}, {'subject_id': 1, 'category_id': 23, 'object_id': -1}, {'subject_id': 2, 'category_id': 23, 'object_id': -1}], 'annotations': [{'category_id': 19, 'bbox': [73.59, 392.83, 393.86, 481.96], 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'category_id': 1, 'bbox': [0.0, 128.83, 160.64, 476.15999999999997], 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'category_id': 1, 'bbox': [175.44, 165.82, 335.72, 401.9], 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'category_id': 19, 'bbox': [127.03, 328.95, 505.86, 483.0], 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'category_id': 31, 'bbox': [51.84, 269.06, 134.53, 433.85], 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'category_id': 31, 'bbox': [267

KeyError: 'orig_size'

: 